In [2]:
import pandas as pd
import bz2
import json
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np
import time
import datetime

In [3]:
features = pd.read_csv('D:/coursera/13 dota/features.csv', index_col='match_id')

In [4]:
df = pd.DataFrame(features)

In [5]:
features = features.drop(['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant',
                         'barracks_status_dire'], axis = 1)

In [6]:
features.fillna(0, inplace=True)

In [20]:
y = features['radiant_win']
X = features.drop(['radiant_win'], axis=1)

In [21]:
kf = KFold(y.size, n_folds=5, random_state=42, shuffle=True)

def accuracy_test_GBM(X, y, kf):
    scores = []
    n_trees = {10, 30, 50, 100}
    for n, tree in enumerate(n_trees):
        start_time = datetime.datetime.now()
        model = GradientBoostingClassifier(random_state=42, n_estimators=tree)
        score = np.mean(cross_val_score(model, X, y, scoring='roc_auc', cv=kf, n_jobs=-1))
        scores.append(np.mean(score))
        print('Time elapsed:', datetime.datetime.now() - start_time, 'Score: ', score, 'Tree number:', tree)  

In [ ]:
accuracy_test_GBM(X, y, kf)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def accuracy_test_LR(X, y, kf):
    scores = list()
    c_range = [0.001,0.01,0.1,1,10,100]
    for c in c_range:
     start_time = datetime.datetime.now()
     model = LogisticRegression(random_state=42, C=c)
     score = np.mean(cross_val_score(model, X, y, scoring='roc_auc', cv=kf, n_jobs=-1))
     scores.append(score)
     time_spent = datetime.datetime.now() - start_time
     print (time_spent, c, score)

In [33]:
kf = KFold(y.size, n_folds=5, random_state=1, shuffle=True)
log_res_scaled = accuracy_test_LR(X_scaled, y, kf)

0:00:10.624985 0.001 0.7162096360086553
0:00:13.015610 0.01 0.7163757959125769
0:00:13.698496 0.1 0.7163509299301707
0:00:14.018305 1 0.7163472703068166
0:00:13.309322 10 0.7163470734968606
0:00:13.657957 100 0.7163470756461708


In [ ]:
X_no_categorical = features.drop(columns=['radiant_win', 'lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero',
                                 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

In [60]:
X_no_categorical.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [ ]:
scaler = StandardScaler()
X_no_categorical_scaled = scaler.fit_transform(X_no_categorical)

In [36]:
kf = KFold(y.size, n_folds=5, random_state=1, shuffle=True)
log_res_no_categorical_scaled = accuracy_test_LR(X_no_categorical_scaled, y, kf)

0:00:09.383481 0.001 0.7162333103715237
0:00:11.421853 0.01 0.7164088702736156
0:00:12.578109 0.1 0.716383801538205
0:00:12.015611 1 0.7163803390297804
0:00:15.191198 10 0.7163798497335823
0:00:14.492654 100 0.7163796823834382


In [ ]:
heroes = pd.read_csv('D:/coursera/13 dota/dictionaries/heroes.csv')

In [ ]:
len(heroes)

In [43]:
def bag_of_words(X):
    X_pick = np.zeros((X.shape[0], 112))
    for i, match_id in enumerate(X.index):
        for p in range(5):
            X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        return pd.DataFrame(X_pick, index=X.index)

In [ ]:
data = bag_of_words(X)

In [23]:
with_bag_of_words = X_no_categorical.join(data)
kf = KFold(y.size, n_folds=5, random_state=42, shuffle=True)
log_res_with_bag_of_words = accuracy_test_LR(with_bag_of_words, y, kf)

0:00:04.089079 0.001 0.5134476155469752
0:00:04.436574 0.01 0.5134476155469752
0:00:04.141162 0.1 0.5134476155469752
0:00:04.183818 1 0.5134476155469752
0:00:04.272225 10 0.5134476155469752
0:00:04.378856 100 0.5134476155469752


In [25]:
with_bag_of_words.shape

(97230, 203)

In [45]:
scaler = StandardScaler()
with_bag_of_words_scaled = scaler.fit_transform(with_bag_of_words)

In [50]:
kf = KFold(y.size, n_folds=5, random_state=42, shuffle=True)
log_res_with_bag_of_words_scaled = accuracy_test_LR(with_bag_of_words_scaled, y, kf)

0:00:11.312488 0.001 0.7163681980680231
0:00:13.361891 0.01 0.7165508270233065
0:00:14.309145 0.1 0.7165263694592239
0:00:14.406221 1 0.7165228167563229
0:00:14.296858 10 0.7165221409201695
0:00:14.937482 100 0.716522132461778


In [91]:
features_test = pd.read_csv('D:/coursera/13 dota/features_test.csv', index_col='match_id')

In [92]:
features_test.fillna(0, inplace=True)

In [93]:
X_test = pd.DataFrame(features_test)

In [94]:
X_pick = np.zeros((X_test.shape[0], 112))

In [ ]:
for i, match_id in enumerate(X_test.index):
        for p in range(5):
            X_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [96]:
heroBOW_test = pd.DataFrame(X_pick, index=X_test.index)

In [97]:
X_test = X_test.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero',
                                 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

In [98]:
with_bag_of_words_test = X_test.join(heroBOW_test)

In [158]:
model = LogisticRegression(random_state=42, C=0.01, n_jobs=-1, class_weight='balanced')

In [ ]:
model.fit(with_bag_of_words, y)

In [160]:
predicted = pd.DataFrame(model.predict(with_bag_of_words_test))
predicted_proba = pd.DataFrame(model.predict_proba(with_bag_of_words_test) [:, 1])

In [166]:
max(predicted_proba[0])

0.9791659433931096

In [167]:
min(predicted_proba[0])

0.0031253623235126546